In [1]:
!pip install wandb

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -- ------------------------------------- 0.1/2.2 MB 3.6 MB/s eta 0:00:01
   ---- ----------------------------------- 0.3/2.2 MB 3.2 MB/s eta 0:00:01
   ------- -------------------------------- 0.4/2.2 MB 2.6 MB/s eta 0:00:01
   -------------- ------------------------- 0.8/2.2 MB 3.8 MB/s eta 0:00:01
   ------------------ --------------------- 1.0/2.2 MB 4.0 MB/s eta 0:00:01
   ------------------------- -------------- 1.4/2.2 MB 4.8 MB/s eta 0:00:01
   ------------------------------- -------- 1.7/2.2 MB 4.9 MB/s eta 0:00:01
   ------------------------------------- -- 2.0/2.2 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 5.3 MB/s eta 0:00:00
Using cached click-8.1.7-py3-none-any.whl (97 kB)
   -------


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import wandb
from models import inception_resnet_v1
from models.utils.dataset_generator import generate_dataset
from models.utils.dataset import TripletFaceDataset
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from models.utils import embed_train
from models.mtcnn import MTCNN

In [11]:
wandb.login()

config={
    "learning_rate": 0.001,
    "momentum": 0.9,
    "epochs": 10,
    "batch_size":32,
    "optimizer": "sgd",
    "margin": 2.5,
    "image_size": 128
}

run = wandb.init(
    project="FaceNet",
    config=config
)

config = run.config

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [12]:
dataset_df = generate_dataset("E:\\105_classes_pins_dataset")

In [13]:
transform = v2.Compose([
    v2.RandomAutocontrast(p=0.13),
    v2.RandomAdjustSharpness(2, 0.12)
])

dataset = TripletFaceDataset(
    triplets_dataframe=dataset_df,
    cropper=MTCNN(config.image_size),
    transform=transform
)

dataloader = DataLoader(
    dataset=dataset,
    batch_size=config.batch_size,
    num_workers=1
) 

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = inception_resnet_v1.InceptionResnetV1(
    pretrained='vggface2',
    device=device
)

In [15]:
for i in range(config.epochs):
    model, loss = embed_train.train_one_epoch(
        model=model,
        optimizer=config.optimizer,
        lr=config.learning_rate,
        momentum=config.momentum,
        margin=config.margin,
        train_loader=dataloader,
        epoch=i,
        log_interval=5,
        device=device,
        run=run
    )

	Batch: 0, Loss: 0.3335
	Batch: 5, Loss: 1.2759
	Batch: 10, Loss: 0.5045
	Batch: 15, Loss: 0.1152
	Batch: 20, Loss: 0.0897
	Batch: 25, Loss: 0.0465
	Batch: 30, Loss: 0.0300


KeyboardInterrupt: 